# Coursera Capstone
### This notebook will be mainly used for the capstone project

In [55]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from bs4 import BeautifulSoup
import requests
import json

CLIENT_ID = 'C5P10I52RGGI1IVOHJGQH0YYVDHATZPIOVIFKULYJOA5TDKQ' # your Foursquare ID
CLIENT_SECRET = 'QCXHK0JKWFGCT2S5OXWZJARHX3DYJMVIX2ZXEFXXSFBCTAOS' # your Foursquare Secret
VERSION = '20180602'
LIMIT = 30

In [56]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Get the list of US universities

In [57]:
# Parse list universities
url = "https://www.latlong.net/category/universities-236-47.html"
soup = BeautifulSoup(requests.get(url).text, "html.parser")
table = soup.find("table")
# Create dataframe universities
df_uni = pd.read_html(str(table))[0]
df_uni.columns = ['name', 'lat', 'lng']
df_uni['name'] = df_uni['name'].str.split(',').str[0]

#### Get the list of venues for each university

In [116]:
search_query = 'restaurant'
radius = 500

# Define the dataframe columns
column_names = ['id', 'university', 'name', 'category', 'price_category', 'rating', 'lat', 'lng'] 
# instantiate the dataframe
df_venues = pd.DataFrame(columns=column_names)

limit_uni = 15
nb = 0
# For each university
for index, row in df_uni.iterrows():
    nb = nb + 1 
    if nb < limit_uni: 
        latitude = row['lat']
        longitude = row['lng']
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
        response = requests.get(url).json()['response']
        if 'venues' in response:
            venues_data = response['venues']
             # For each venue
            for data in venues_data:
                id = data['id']
                name = data['name']
                if len(data['categories']) > 0:
                    category = data['categories'][0]['name']
                else :
                    category = '' ;
                #lat = data['location']['lat']
                #lng = data['location']['lng']
                url_venue = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id, CLIENT_ID, CLIENT_SECRET, VERSION)
                #venue_details = requests.get(url_venue).json()['response']

                if 'rating' in venue_details:
                    rating = venue_details['rating']
                else :
                    rating = '2'

                if 'price' in venue_details:
                    price_category = venue_details['price']['catrgory'] #a changer
                else :
                    price_category = '1'

                df_venues = df_venues.append({'name': name, 'category': category, 'price_category' : price_category, 'rating' : rating, 'university': row['name']}, ignore_index=True)

# Drop empty columns
df_venues['category'].replace('', np.nan, inplace=True)
df_venues['price_category'].replace('', np.nan, inplace=True)
df_venues['rating'].replace('', np.nan, inplace=True)
df_venues.dropna(subset=['category'], inplace=True)

df_venues = df_venues.groupby(['university','category']).size().unstack()

df_venues.reset_index(inplace=True)
df_venues = df_venues.set_index('university')
df_venues = df_venues.fillna(0)
df_venues['total'] = df_venues.sum(axis=1)
df_venues.head()

category,American Restaurant,Asian Restaurant,Bakery,Bar,Breakfast Spot,Chinese Restaurant,Diner,Food,Gastropub,Greek Restaurant,...,Other Nightlife,Pizza Place,Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,total
university,,,,,,,,,,,,,,,,,,,,,
Harvard University,2.0,1.0,0.0,2.0,1.0,2.0,0.0,3.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,22.0
Indiana University of Pennsylvania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Johns Hopkins University,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
Lawrence University,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Massachusetts Institute of Technology,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0


#### KMeans

In [117]:
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 4, random_state = 0).fit(df_venues)

In [118]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = df_venues.columns
means_df.index = ['1', '2', '3', '4']
means_df['total'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['total'], ascending=False)

category,American Restaurant,Asian Restaurant,Bakery,Bar,Breakfast Spot,Chinese Restaurant,Diner,Food,Gastropub,Greek Restaurant,...,Other Nightlife,Pizza Place,Restaurant,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,total
4,2.0,1.0,0.0,2.0,1.0,2.0,0.0,3.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,44.0
2,0.0,1.0,0.0,1.0,0.5,2.5,0.5,0.5,0.0,0.5,...,0.0,0.5,0.5,0.0,2.0,0.0,0.5,0.5,0.0,28.0
1,1.5,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,15.0
3,0.4,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.4,0.0,0.2,0.0,0.0,0.0,0.0,3.2


#### Assign group to each university

In [119]:
uni_smry = pd.DataFrame([df_venues.index, 1 + kmeans.labels_]).T
uni_smry.columns = ['university', 'group']
uni_smry

,university,group
0,Harvard University,4
1,Indiana University of Pennsylvania,3
2,Johns Hopkins University,3
3,Lawrence University,3
4,Massachusetts Institute of Technology,1
5,Penn State University,1
6,University of Chicago,3
7,University of Illinois,2
8,University of Michigan,2
9,Utah Valley University,3
